In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import arviz as az
from cmdstanpy import cmdstan_path, CmdStanModel
import json
from tqdm.notebook import tqdm
import seaborn as sns
import os
from IPython.display import display, HTML
import glob


c:\Users\srboval1\AppData\Local\anaconda3\envs\stan\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data parser

Find and download all microrheology data and format dictionary for stan

In [ ]:
# Download data and make a mega .csv fiel

paths=[]
paths = glob.glob(os.path.join('C:/Users/srboval1/IPN15/AlgVLVG15_3Ca/*','**','summary_ref_level.csv'))
print(paths)
concentration = '3/2'
type ='IPN_soft'

a = []
for i in paths:
    path1 = os.path.split(os.path.split(i)[0])[0]
    path1.replace("/","\\")
    splitted= path1.split('\\')[-1]
    tmp = pd.read_csv(i)
    tmp['coating_type'] = splitted.split('_')[-1]
    tmp['size'] = splitted.split('_')[-2]
    tmp['day'] = splitted.split('_')[-4]
    a.append(tmp)

a_concantenated=[]
a_concantenated = pd.concat(a)
a_concantenated['radius_(m)'] *= 1e6
a_concantenated = a_concantenated.rename(columns={'radius_(m)':'radius_(um)'})
a_concantenated['concentration'] = concentration
a_concantenated['type'] = type
a_concantenated['frequency'] = 0.05
a_concantenated = a_concantenated.reindex(columns=['day','frequency','concentration','type','sample','holder','location','repeat','track_id','reference_id','distance(um)','Cov_Sum','a_(um)','phi_(rad)','c','d','G_abs','radius_(m)','r2','rmse','inv.rmse','shift_(s)','a_error','phi_error','c_error','d_error','x','y','phi_(deg)','tan_phi'])
print(a_concantenated)
#%%
if os.path.exists('C:/Users/srboval1/IPN15/IPN15.csv'):
    a_new = pd.DataFrame(a_concantenated)
    a_new.to_csv('C:/Users/srboval1/IPN15/IPN15.csv', mode='a', index=False, header=False)
else:
    a_concantenated.to_csv("C:/Users/srboval1/IPN15/IPN15.csv", index=False)
    

['C:/Users/srboval1/IPN15/AlgVLVG15_3Ca\\230728_3_10_plain\\results\\summary_ref_level.csv', 'C:/Users/srboval1/IPN15/AlgVLVG15_3Ca\\230814_3_10_plain\\results\\summary_ref_level.csv', 'C:/Users/srboval1/IPN15/AlgVLVG15_3Ca\\230815_3_10_plain\\results\\summary_ref_level.csv']
        day  frequency concentration      type  sample  holder  location  \
0    230728       0.05           3/2  IPN_soft       1       1         1   
1    230728       0.05           3/2  IPN_soft       1       1         1   
2    230728       0.05           3/2  IPN_soft       1       1         1   
3    230728       0.05           3/2  IPN_soft       1       1         1   
4    230728       0.05           3/2  IPN_soft       1       1         1   
..      ...        ...           ...       ...     ...     ...       ...   
169  230815       0.05           3/2  IPN_soft       1       3         3   
170  230815       0.05           3/2  IPN_soft       1       3         3   
171  230815       0.05           3/2  I

In [2]:
data_all = pd.read_csv('C:/Users/srboval1/IPN15/datas.csv')

#calculating G std for each material typeand then choosing the highest one of all
G_std = np.max(data_all[['type','day','holder','location','track_id',"G_abs", "phi_(deg)"]].groupby(["type"]).std()["G_abs"].values)
print(G_std)

#calculating G std for each material type and then choosing the highest one of all
phi_std = np.max(data_all[['type','day','holder','location','track_id',"G_abs", "phi_(deg)"]].groupby(["type"]).std()["phi_(deg)"].values)
print(phi_std)


full_path = os.path.split(os.getcwd())[0]

print(["G_abs", "phi_(rad)"], "\n",data_all["type"].unique())

687.3652925606983
8.000944348706229
['G_abs', 'phi_(rad)'] 
 ['collagen' 'IPN_tuned']


This python code which includes model definition, which itself is written in Stan language, while Python is used to preprocess data, compile the model, run inference, and analyze results.

1. Creating a dictionary from the collected data - preprocessing
2. Compiling the model, sampling it on the data
3. Interference and analysis - fitting

In [25]:
#function preprocessing data and fitting model
#model_name: name of the stan model e.g hier_material (no .stan)
#diagnoise: print mcmc diagnostics, rhat...
#returns: tuple of two dicts where keys are G_abs and phi_(deg)
#first containts the mcmc samples as pandas dataframe
#seconds contains arviz datastructure for visualizations
def preprocessing(model_name, diagnose = False):
        
    out = {}
    az_out = {}

    data_all = pd.read_csv('C:/Users/srboval1/IPN15/datas.csv')

    #process G and phi measurements separately, 
    for measurement in ['G_abs','phi_(deg)']:
        samples = []
        out[measurement] = {}

        #combine information of all materials
        #looping through each material one by one
        for material in data_all["type"].unique():
            data_type = material
            data = data_all[data_all['type'] == data_type]

            #initializing empty list that will contain corresponding numbers for each sample
            sample_num = []
        
            prev = None
            idx = 1
            #assuming one sample per day, adding sample ids to the list in sample_num
            #iterating over each element in the 'day' column of data and creating the sample_num list
            for i in data['day']:
                if prev is None:
                    prev = i
                else: #changed from Ossi's code 'if' - to have distinct sample ids
                    idx += 1
                    prev = i
                sample_num.append(idx)      
            
            data['sample_num'] = sample_num
            subset = data [['sample_num','holder','location', 'track_id', measurement]]
            subset['sample_num']=subset['sample_num'].astype(int) #convertin sample numbers to integers
            
            sample_ids=[]
            location_ids=[]
            holder_ids=[]
            track_ids=[]

            n_loc = 0       #initial number of ids in locations
            n_hold = 0      #initial locations in holders
            n_samp = 0    #initial holders in samples

        #change into consecutive numbering
            for i in subset['sample_num'].unique():             
                sub_sample = subset[subset['sample_num']==i]        #new df containing only the rows (measurements) with that unique sample_num i
                sample_ids.extend([int(i)]*sub_sample['holder'].unique().shape[0])
                for jj,j in enumerate(sub_sample['holder'].unique()):
                    sub = sub_sample[sub_sample['holder']==j]
                    holder_ids.extend([int(jj+1)+n_samp]*sub['location'].unique().shape[0]) #list that will eventually be listing all holder ids for each location - add identity number of holder times the number of locations (unique holder numbers)
                    for kk,k in enumerate(sub['location'].unique()):
                        sub2 = sub[sub['location']==k]
                        location_ids.extend([int(kk+1)+n_hold]*sub2['track_id'].unique().shape[0])
                        for zz, z in enumerate(sub2['track_id'].unique()):
                            sub3 = sub2[sub2['track_id']==z]
                            track_ids.extend([int(zz+1)+n_loc]*sub3.shape[0]) #shape[0] size of the first dimension = row
                        n_loc += int(sub2['track_id'].unique().shape[0])    #number of tracks ids that have been looped through already; updated with each location
                    n_hold += int(sub['location'].unique().shape[0])    #number of locations that have been looped through; used to add up in the next holder
                n_samp += int(sub_sample['holder'].unique().shape[0])   #number of holders that have been looped through; used to add up in the next sample

        # normalize mean to zero and with common std 
        y_raw = subset[measurement].values  #retrieves values of the column labelled measurement from df subset and converts into array by .value
        if measurement == 'G_abs':          #compares measurement to a fixed string 'G_abs' variable
            y = y_raw/G_std                 #each value scaled by std
        else:
            y = y_raw/phi_std

        #defining dictionary; curly brackets and specified key-value pairs separated by : each key corresponds to a specific attribute }
        sample = {      'N'         :subset.shape[0],
                        'N_samples' : int(np.max(sample_ids)),
                        'N_holders' : int(np.max(holder_ids)),
                        'N_locations':int(np.max(location_ids)),
                        'N_ids'     : int(np.max(track_ids)),
                        'sample_ids': sample_ids,
                        'holder_ids': holder_ids,
                        'location_ids':location_ids,
                        'track_ids':  track_ids,
                        'train_ids':  (np.arange(len(track_ids))+1).tolist(),
                        'N_train':    subset.shape[0],
                        'y':          y.tolist()}
        
        #appending a shallow copy 'samples' of sample - doesn't create copies of the objects it references to
        samples.append(sample.copy())

    # combine different materials into a single dict/json for stan
    combined = samples[0]           #combined is assigned the value of the first element (samples[0]) from the list samples
    combined['material_ids'] = np.ones(combined['N_samples'],dtype=int).tolist() #creates a list of ones with a length determined by the value of 'N_samples'
    s_max = combined['N_samples']
    h_max = combined['N_holders']
    l_max = combined['N_locations']
    t_max = combined['N_ids']
    m_max = 1

    for s in samples[1:]:       #code iterates over all elements in the samples list except the first one
        #print(s['N'])
        combined['N'] += s['N'] #accesses the value associated with the key 'N' in the dictionary combined and incrementing it by the value associated with the key 'N' in each dictionary s in the samples list
        combined['N_samples'] += s['N_samples']
        combined['N_holders'] += s['N_holders']
        combined['N_locations'] += s['N_locations']
        combined['N_ids'] += s['N_ids']
        combined['sample_ids'].extend((np.array(s['sample_ids'])+s_max).tolist())
        combined['holder_ids'].extend((np.array(s['holder_ids'])+h_max).tolist())
        combined['location_ids'].extend((np.array(s['location_ids'])+l_max).tolist())
        combined['track_ids'].extend((np.array(s['track_ids'])+t_max).tolist())
        combined['y'].extend(s['y'])
        combined['material_ids'].extend((np.ones(s['N_samples'],dtype=int)+m_max).tolist())

        s_max += s['N_samples']
        h_max += s['N_holders']
        l_max += s['N_locations']
        t_max += s['N_ids']
        m_max += 1

    combined['train_ids'] = ((np.arange(combined['N'])+1).astype(int)).tolist()
    combined['N_train'] = len(combined['train_ids'])
    combined['sample_ids'] = combined['sample_ids']
    combined['holder_ids'] = combined['holder_ids']
    combined['location_ids'] = combined['location_ids']
    combined['track_ids'] = combined['track_ids']
    combined['material_ids'] = np.array(combined['material_ids']).astype(int).tolist()
    combined['N_materials'] = len(samples)

    import json

    #crating path to which the dictionary is saved in json format
    
    file_path = os.path.join("C:/Users/srboval1/IPN15", f'IPN15_dict_{measurement}.json')

    # Writing the dictionary to a JSON file
    with open(file_path, "w") as json_file:
        json.dump(combined, json_file)

    # Stan model = math of the Bayesian model
    # function CmdStanModel is used to compile the file with Stan model, e.g. converting it into a format that can be executed by the CmdStan command-line tool.
    stan_model_path = 'C:/Users/srboval1/InstruProject/microrheology/models/model_Linda.stan'
    model = CmdStanModel(stan_file=stan_model_path)
    

    # Bayes model: all the available data is used to estimate the parameters of the model
    # and the posterior distribution which is then used for inference and analysis


    #Using sample method of CmdStanModel
    # sampling (all) data from the file path by Hamiltonian Monte Carlo (HMC) algorithm
    # to generate samples from the posterior distribution of the model parameters
    # high adaptation value means adaptation phase is not too aggressive
    # maximum depth of the binary tree used by the algorithm - controls the complexity of the trajectories explored by the sampler
    fit = model.sample(data=file_path,
                            adapt_delta=0.9999999999999999,max_treedepth=20,iter_sampling=2000)
    if diagnose:
        print(fit.diagnose())

    #converting the sampled data from fit into an ArviZ InferenceData object
    #ArviZ (az) library allows exploratory analysis of Bayesian models
    #object is created from the CmdStanPy object and contains posterior samples, posterior predictive samples, log likelihood, and observed data.
    az_data = az.from_cmdstanpy(posterior=fit,
                                posterior_predictive='y_hat',
                                log_likelihood='log_likelihood',
                                observed_data={'y':combined['y']})
    
    #returns the sampled data obtained from the Bayesian model's sampling output in a pandas DataFrame format in a dictionary
    #basic data manipulation and analysis using pandas functionalities
    out[measurement] = fit.draws_pd()

    #additional functionalities specific to Bayesian analysis
    #posterior predictive checks, convergence diagnostics, and visualization tools, through the ArviZ library.
    az_out[measurement] = az_data
    
    return out,az_out

In [26]:
hh,az_out = preprocessing("model_Linda", False)

C:\Users\srboval1\AppData\Local\Temp\ipykernel_30372\2330346121.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sample_num'] = sample_num
C:\Users\srboval1\AppData\Local\Temp\ipykernel_30372\2330346121.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['sample_num']=subset['sample_num'].astype(int) #convertin sample numbers to integers
C:\Users\srboval1\AppData\Local\Temp\ipykernel_30372\2330346121.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

ValueError: Failed to get source info for Stan model 'C:\Users\srboval1\InstruProject\microrheology\models\model_Linda.stan'. Console:
Syntax error in 'C:\Users\srboval1\InstruProject\microrheology\models\model_Linda.stan', line 9, column 18 to column 19, parsing error:
   -------------------------------------------------
     7:    int N_materials;
     8:    int N_ids;
     9:    int material_ids[N_samples];
                           ^
    10:    int sample_ids[N_holders];
    11:    int holder_ids[N_locations];
   -------------------------------------------------

";" expected after variable declaration.
